In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd, gc

In [2]:
rng = np.random.default_rng()

In [3]:
class Layer:
    
    def __init__(self, n_in, n_out, *, actv):
        if actv == "relu": self.W = rng.normal(0, 2/np.sqrt(n_in), size=(n_out, 1+n_in))
        else: self.W = rng.uniform(-(x := np.sqrt(6/(n_in+n_out))), x, size=(n_out, 1+n_in))
        #self.W = rng.uniform(-0.5, 0.5, size=(n_out, 1+n_in))
        self.actv_func = getattr(self, f"{actv}_func")
        self.actv_grad = getattr(self, f"{actv}_grad")
        self.X = None
        self.Z = None
        self.A = None
        self.d_Z = None
        self.d_AZ = None
        self.Z_exp = None
        self.s = None
        self.size = 0
    
    def __call__(self, X):
        return self.W @ np.vstack((np.ones((1, len(X.T)), dtype=np.float64), X))
    
    def forward(self, X):
        n = len(X.T)
        if n > self.size:
            self.X = np.vstack((np.ones((1, n), dtype=np.float64), X))
            self.Z = self.W @ self.X
            self.A = np.zeros(self.Z.shape, dtype=np.float64)
            self.d_Z = np.zeros(self.Z.shape, dtype=np.float64)
            self.d_W = np.zeros(self.W.shape, dtype=np.float64)
            self.d_A = np.zeros(self.Z.shape, dtype=np.float64)
            self.Z_exp = np.zeros(self.Z.shape, dtype=np.float64)
            self.d_AZ = np.zeros((len(self.Z), len(self.Z)), dtype=np.float64)
            self.s = np.zeros((len(self.Z.T), ), dtype=np.float64)
            self.size = n
        else:
            self.X[1:, :n] = X
            np.dot(self.W, self.X[:, :n], out=self.Z[:, :n])
        self.actv_func(n)
        return self.A[:, :n]
    
    def backward(self, d_A):
        n = len(d_A.T)
        self.d_A[:, :n] = d_A
        self.actv_grad(n)
        np.dot(self.d_Z[:, :n], self.X.T[:n, :], out=self.d_W)
        return self.W.T[1:] @ self.d_Z[:, :n]
    
    def relu_func(self, size):
        self.A[:, :size] = self.Z[:, :size] * (self.Z[:, :size] > 0)
    
    def relu_grad(self, size):
        self.d_Z[:, :size] = self.d_A[:, :size] * (self.Z[:, :size] > 0)
    
    def softmax_func(self, size):
        np.exp(np.maximum(-100, np.minimum(100, self.Z[:, :size])), out=self.Z_exp[:, :size])
        np.divide(self.Z_exp[:, :size], self.Z_exp[:, :size].sum(axis=0), out=self.A[:, :size])
    
    def softmax_grad(self, size):
        self.d_Z[:, :size] = self.d_A[:, :size] * self.A[:, :size]
        '''
        self.s[:size] = self.Z_exp[:, :size].sum(axis=0)
        for j, z_exp in enumerate(self.Z_exp.T[:size, :]):
            for i in range(len(z_exp)):
                self.d_AZ[i, i] = (self.s[i] - self.Z_exp[i, j]) * self.Z_exp[i, j]
                for i1 in range(i+1, len(z_exp)):
                    self.d_AZ[i, i1] = self.d_AZ[i1, i] = - (self.Z_exp[i, j] * self.Z_exp[i1, j])
            self.d_AZ /= self.s[j] ** 2
            self.d_Z[:, j] = self.d_AZ[:, :size] @ self.d_A.T[j]
        '''

In [4]:
class NeuNet:
    
    def __init__(self, n_in, ns_out, *, actvs=None, alpha=None, beta=2**-4, tol=0.15, max_iter=256):
        n = len(ns_out)
        in_out = [n_in] + ns_out
        if actvs is None: actvs = ['relu'] * (n - 1) + ['softmax']
        self.layers = [Layer(in_out[i], in_out[i+1], actv=actv) for i, actv in enumerate(actvs)]
        self.alpha = alpha
        self.beta = beta
        self.tol = tol
        self.max_iter = max_iter
    
    def __call__(self, X):
        if not isinstance(X, np.ndarray): X = np.array(X)
        y_p = X = np.atleast_2d(X).T
        for layer in self.layers: y_p = layer(y_p)
        return np.apply_along_axis(np.argmax, 0, y_p)
    
    @property
    def W(self):
        return np.array([layer.W for layer in self.layers])
    
    def one_hot(self, x, maxx=10):
        return np.apply_along_axis(lambda k: (np.arange(maxx)==k).astype(int), 0, np.atleast_2d(x))
    
    def fit(self, X, y):
        if not isinstance(X, np.ndarray): X = np.array(X)
        if not isinstance(y, np.ndarray): y = np.array(y)
        X = np.atleast_2d(X).T
        y_hot = self.one_hot(y)
        if self.alpha is not None: alpha, d_alpha = self.alpha, 0
        else: alpha, d_alpha = 1, (1 - self.beta) / self.max_iter
        for itr in range(self.max_iter):
            y_p = X
            for layer in self.layers: y_p = layer.forward(y_p)
            d_y = (y_p - y_hot) / len(y)
            for layer in reversed(self.layers):
                d_y = layer.backward(d_y)
                layer.W -= alpha * layer.d_W
            alpha -= d_alpha
            loss = np.sum((y_hot - y_p) ** 2, axis=0).mean()
            acc = np.mean(np.apply_along_axis(np.argmax, 0, y_p) == y)
            print(f"{itr = }\t{loss = :.6f}\tacc = {100*acc:.3f}%".expandtabs(10))
            if loss < self.tol: break
        return self
    
    def __repr__(self): return f"NeuNet({len(self.layers)} layers)"

In [5]:
df = pd.read_csv("train.csv")
df.shape

(42000, 785)

In [6]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data = df.to_numpy().astype(np.float64)
X = data.T[1:].T / 256
y = data.T[0]
X.shape, y.shape

((42000, 784), (42000,))

In [8]:
X_trn = X[:30_000]
y_trn = y[:30_000]
X_tst = X[30_000:]
y_tst = y[30_000:]

In [13]:
model = NeuNet(len(X.T), [16, 10], beta=0.1)

In [14]:
model.fit(X_trn, y_trn)

itr = 0   loss = 0.933918     acc = 8.960%
itr = 1   loss = 0.914784     acc = 12.660%
itr = 2   loss = 0.901289     acc = 16.323%
itr = 3   loss = 0.888645     acc = 19.613%
itr = 4   loss = 0.875556     acc = 22.167%
itr = 5   loss = 0.861685     acc = 23.547%
itr = 6   loss = 0.847793     acc = 24.530%
itr = 7   loss = 0.834582     acc = 25.603%
itr = 8   loss = 0.821753     acc = 26.903%
itr = 9   loss = 0.809350     acc = 28.027%
itr = 10  loss = 0.797340     acc = 29.297%
itr = 11  loss = 0.785339     acc = 30.720%
itr = 12  loss = 0.773219     acc = 32.543%
itr = 13  loss = 0.760698     acc = 34.810%
itr = 14  loss = 0.747783     acc = 37.363%
itr = 15  loss = 0.734292     acc = 40.037%
itr = 16  loss = 0.719831     acc = 42.397%
itr = 17  loss = 0.704378     acc = 44.657%
itr = 18  loss = 0.687869     acc = 46.550%
itr = 19  loss = 0.670716     acc = 48.430%
itr = 20  loss = 0.652873     acc = 50.107%
itr = 21  loss = 0.635256     acc = 51.373%
itr = 22  loss = 0.617596     acc

itr = 187 loss = 0.157840     acc = 89.293%
itr = 188 loss = 0.157744     acc = 89.300%
itr = 189 loss = 0.157650     acc = 89.307%
itr = 190 loss = 0.157557     acc = 89.327%
itr = 191 loss = 0.157466     acc = 89.340%
itr = 192 loss = 0.157377     acc = 89.340%
itr = 193 loss = 0.157288     acc = 89.350%
itr = 194 loss = 0.157201     acc = 89.353%
itr = 195 loss = 0.157117     acc = 89.350%
itr = 196 loss = 0.157034     acc = 89.350%
itr = 197 loss = 0.156952     acc = 89.360%
itr = 198 loss = 0.156872     acc = 89.347%
itr = 199 loss = 0.156793     acc = 89.347%
itr = 200 loss = 0.156716     acc = 89.357%
itr = 201 loss = 0.156639     acc = 89.357%
itr = 202 loss = 0.156564     acc = 89.353%
itr = 203 loss = 0.156490     acc = 89.350%
itr = 204 loss = 0.156417     acc = 89.367%
itr = 205 loss = 0.156346     acc = 89.370%
itr = 206 loss = 0.156275     acc = 89.377%
itr = 207 loss = 0.156206     acc = 89.380%
itr = 208 loss = 0.156138     acc = 89.383%
itr = 209 loss = 0.156071     ac

NeuNet(2 layers)

In [16]:
np.mean(model(X_tst)==y_tst)

0.8885